In [1]:
%load_ext sql
%sql sqlite:///chinook.db

## Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (т.е. треков куплено больше, чем в других странах).

In [2]:
%%sql
SELECT BillingCountry, COUNT(*) as TotalSales
FROM invoices
JOIN invoice_items ON invoices.InvoiceId = invoice_items.InvoiceId
JOIN tracks ON invoice_items.TrackId = tracks.TrackId
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN artists ON albums.ArtistId = artists.ArtistId
WHERE artists.Name = 'Iron Maiden'
GROUP BY BillingCountry
ORDER BY TotalSales DESC
LIMIT 1;

 * sqlite:///chinook.db
Done.


BillingCountry,TotalSales
USA,34


## Задание 2
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом *(цена альбома - сумма цен треков)*.

In [7]:
%%sql
WITH AlbumPrices AS (
    SELECT 
        albums.AlbumId,
        SUM(tracks.UnitPrice) AS TotalPrice
    FROM 
        albums
    JOIN tracks ON albums.AlbumId = tracks.AlbumId
    GROUP BY albums.AlbumId
),
AveragePrice AS (
    SELECT AVG(TotalPrice) AS AvgPrice FROM AlbumPrices
),
AlbumDurations AS (
    SELECT 
        albums.AlbumId, 
        albums.Title, 
        SUM(tracks.Milliseconds) / 60000.0 AS TotalDurationMinutes
    FROM 
        albums
    JOIN tracks ON albums.AlbumId = tracks.AlbumId
    GROUP BY albums.AlbumId
)
SELECT 
    AlbumDurations.Title, 
    AlbumDurations.TotalDurationMinutes
FROM 
    AlbumDurations
JOIN AlbumPrices ON AlbumDurations.AlbumId = AlbumPrices.AlbumId
CROSS JOIN AveragePrice
WHERE 
    AlbumPrices.TotalPrice > AveragePrice.AvgPrice
ORDER BY 
    AlbumDurations.TotalDurationMinutes DESC
LIMIT 3;


 * sqlite:///chinook.db
Done.


Title,TotalDurationMinutes
"Lost, Season 3",1177.7597
"Battlestar Galactica (Classic), Season 1",1170.2297333333333
"Lost, Season 1",1080.9156


## Задание 3
Вывести общую стоимость треков, приобретённых компаниями (только клиентами из конкретных компаний), которые обслуживались сотрудниками, нанятыми после 3 августа 2002 года.

In [9]:
%%sql
WITH HiredAfter2002 AS (
    SELECT EmployeeId
    FROM employees
    WHERE HireDate > '2002-08-03'
),
CustomersOfHiredEmployees AS (
    SELECT CustomerId
    FROM customers
    WHERE SupportRepId IN (SELECT EmployeeId FROM HiredAfter2002) AND Company IS NOT NULL
)
SELECT 
    SUM(invoice_items.UnitPrice * invoice_items.Quantity) AS TotalCost
FROM invoice_items
JOIN invoices ON invoice_items.InvoiceId = invoices.InvoiceId
WHERE invoices.CustomerId IN (SELECT CustomerId FROM CustomersOfHiredEmployees);


 * sqlite:///chinook.db
Done.


TotalCost
230.72000000000062


## Задание 4
Для каждого альбома определить все жанры песен, которые в нём есть и количество песен по каждому жанру (формат вывода: альбом, жанр, количество).
Выводить только те альбомы, где жанров больше одного.

In [11]:
%%sql
WITH AlbumGenres AS (
    SELECT 
        albums.Title AS AlbumTitle,
        genres.Name AS GenreName,
        COUNT(tracks.TrackId) AS TrackCount,
        albums.AlbumId
    FROM 
        albums
    JOIN tracks ON albums.AlbumId = tracks.AlbumId
    JOIN genres ON tracks.GenreId = genres.GenreId
    GROUP BY albums.AlbumId, genres.GenreId
),
AlbumGenreCounts AS (
    SELECT 
        AlbumId, 
        COUNT(DISTINCT GenreName) AS GenreCount
    FROM AlbumGenres
    GROUP BY AlbumId
    HAVING GenreCount > 1
)
SELECT 
    ag.AlbumTitle, 
    ag.GenreName, 
    ag.TrackCount
FROM 
    AlbumGenres ag
JOIN AlbumGenreCounts agc ON ag.AlbumId = agc.AlbumId
ORDER BY 
    ag.AlbumTitle, 
    ag.TrackCount DESC;


 * sqlite:///chinook.db
Done.


AlbumTitle,GenreName,TrackCount
"Battlestar Galactica, Season 3",Science Fiction,12
"Battlestar Galactica, Season 3",TV Shows,5
"Battlestar Galactica, Season 3",Sci Fi & Fantasy,2
Greatest Hits,Rock,30
Greatest Hits,Metal,14
Greatest Hits,Reggae,13
"Heroes, Season 1",Drama,20
"Heroes, Season 1",TV Shows,3
"LOST, Season 4",Drama,14
"LOST, Season 4",TV Shows,3


## Задание 5
Для каждого артиста (таблицы Artists), у которых есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение суммы, заработанной артистом по этому жанру к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, максимальное для артиста, то выбирается жанр раньше по алфавиту. Формат вывода: артист, жанр, соотношение.

In [12]:
%%sql
WITH ArtistGenreEarnings AS (
    SELECT 
        artists.ArtistId,
        artists.Name AS ArtistName,
        genres.Name AS GenreName,
        SUM(invoice_items.UnitPrice * invoice_items.Quantity) AS GenreEarnings
    FROM 
        invoice_items
    JOIN tracks ON invoice_items.TrackId = tracks.TrackId
    JOIN albums ON tracks.AlbumId = albums.AlbumId
    JOIN artists ON albums.ArtistId = artists.ArtistId
    JOIN genres ON tracks.GenreId = genres.GenreId
    GROUP BY artists.ArtistId, genres.Name
),
TotalEarnings AS (
    SELECT 
        ArtistId,
        SUM(GenreEarnings) AS TotalArtistEarnings
    FROM 
        ArtistGenreEarnings
    GROUP BY ArtistId
),
MaxEarnings AS (
    SELECT 
        ArtistId,
        MAX(GenreEarnings) AS MaxEarnings
    FROM 
        ArtistGenreEarnings
    GROUP BY ArtistId
),
ArtistMaxGenre AS (
    SELECT 
        ArtistGenreEarnings.ArtistId,
        ArtistGenreEarnings.ArtistName,
        ArtistGenreEarnings.GenreName,
        ArtistGenreEarnings.GenreEarnings
    FROM 
        ArtistGenreEarnings
    JOIN MaxEarnings ON ArtistGenreEarnings.ArtistId = MaxEarnings.ArtistId 
        AND ArtistGenreEarnings.GenreEarnings = MaxEarnings.MaxEarnings
),
ArtistGenreCount AS (
    SELECT 
        ArtistId, 
        COUNT(DISTINCT GenreName) AS GenreCount
    FROM 
        ArtistGenreEarnings
    GROUP BY ArtistId
    HAVING GenreCount > 1
)
SELECT 
    ag.ArtistName,
    ag.GenreName,
    ROUND((ag.GenreEarnings / te.TotalArtistEarnings), 2) AS EarningsRatio
FROM 
    ArtistMaxGenre ag
JOIN TotalEarnings te ON ag.ArtistId = te.ArtistId
JOIN ArtistGenreCount agc ON ag.ArtistId = agc.ArtistId
ORDER BY 
    ag.ArtistName, 
    ag.GenreName;


 * sqlite:///chinook.db
Done.


ArtistName,GenreName,EarningsRatio
Amy Winehouse,R&B/Soul,0.67
Antônio Carlos Jobim,Latin,0.68
Audioslave,Rock,0.38
Battlestar Galactica,Science Fiction,0.5
Eric Clapton,Blues,0.68
Faith No More,Alternative & Punk,0.79
Foo Fighters,Rock,0.77
Gilberto Gil,Latin,0.65
Guns N' Roses,Rock,0.72
Heroes,Drama,0.85
